In [47]:
# First import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [48]:
# Load the datasets
data = pd.read_csv("D:/regression_project/Cab_data/cab_rides.csv")
data1 = pd.read_csv("D:/regression_project/Cab_data/weather.csv",sep=',')

# Create copies of the datasets
cb = data.copy()
we = data1.copy()

In [49]:
cb.head(1)

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name
0,0.44,Lyft,1544952607890,North Station,Haymarket Square,5.0,1.0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,lyft_line,Shared


In [50]:
we.head(1)

,temp,location,clouds,pressure,rain,time_stamp,humidity,wind
0,42.42,Back Bay,1.0,1012.14,0.1228,1545003901,0.77,11.25


In [51]:
# Check the length of the time_stamp rows for cb
len(str(cb.loc[1,"time_stamp"]))

13

In [52]:
# Check the length of the time_stamp rows for we
len(str(we.loc[1,"time_stamp"]))

10

In [53]:
# convert time_stamp to datetime and convert cb milli sec to sec
cb['date_time'] = pd.to_datetime(cb['time_stamp']/1000, unit='s')
we['date_time'] = pd.to_datetime(we['time_stamp'], unit='s')


In [54]:
cb['date_time'].unique()

<DatetimeArray>
['2018-12-16 09:30:07.890000105', '2018-11-27 02:00:23.677000046',
 '2018-11-28 01:00:22.197999954', '2018-11-30 04:53:02.749000072',
 '2018-11-29 03:49:20.223000050', '2018-12-17 18:25:12.138000011',
 '2018-11-26 05:03:00.200000048', '2018-12-02 19:53:04.677000046',
 '2018-12-03 06:28:02.644999981', '2018-11-27 10:45:22.249000072',
 ...
 '2018-12-13 18:15:03.572999954', '2018-11-29 17:38:01.897000074',
 '2018-12-03 03:28:02.279000044', '2018-12-15 15:15:13.191999912',
 '2018-12-14 03:05:08.323999882', '2018-11-28 00:15:22.509999989',
 '2018-11-29 01:03:40.023000002', '2018-11-29 01:47:08.122999907',
 '2018-12-02 05:28:04.148999929', '2018-12-01 23:53:05.533999919']
Length: 114335, dtype: datetime64[ns]

In [55]:
we['date_time'].unique()

<DatetimeArray>
['2018-12-16 23:45:01', '2018-11-27 19:45:20', '2018-11-27 00:17:13',
 '2018-11-29 00:02:07', '2018-12-14 11:45:01', '2018-11-26 17:35:10',
 '2018-12-03 09:52:59', '2018-12-15 10:45:01', '2018-12-14 18:45:01',
 '2018-12-17 17:45:01',
 ...
 '2018-11-28 21:02:09', '2018-12-02 22:52:54', '2018-11-26 09:05:11',
 '2018-11-26 06:40:30', '2018-11-30 11:52:54', '2018-12-01 09:52:54',
 '2018-12-17 13:45:02', '2018-12-02 11:52:54', '2018-12-17 22:45:06',
 '2018-12-03 06:52:54']
Length: 840, dtype: datetime64[ns]

In [56]:
cb['source'].count()

np.int64(693071)

In [57]:
we['location'].count()

np.int64(6276)

In [58]:
cb['source'].unique()

array(['Haymarket Square', 'Back Bay', 'North End', 'North Station',
       'Beacon Hill', 'Boston University', 'Fenway', 'South Station',
       'Theatre District', 'West End', 'Financial District',
       'Northeastern University'], dtype=object)

In [59]:
we['location'].unique()

array(['Back Bay', 'Beacon Hill', 'Boston University', 'Fenway',
       'Financial District', 'Haymarket Square', 'North End',
       'North Station', 'Northeastern University', 'South Station',
       'Theatre District', 'West End'], dtype=object)

In [60]:
# Create m_date column in both datasets for merging
cb['m_date'] = cb['source'].astype(str) + "-" + cb['date_time'].dt.date.astype(str) + "-" + cb['date_time'].dt.hour.astype(str)
we['m_date'] = we['location'].astype(str) + "-" + we['date_time'].dt.date.astype(str) + "-" + we['date_time'].dt.hour.astype(str)

In [61]:
# drop duplicates based on m_date from we
we = we.drop_duplicates(subset='m_date')

In [62]:
# Merge the two datasets on m_date
cab = cb.merge(we,on='m_date', how='left')

In [63]:
cab.count()

distance            693071
cab_type            693071
time_stamp_x        693071
destination         693071
source              693071
price               637976
surge_multiplier    693071
id                  693071
product_id          693071
name                693071
date_time_x         693071
m_date              693071
temp                690107
location            690107
clouds              690107
pressure            690107
rain                 98756
time_stamp_y        690107
humidity            690107
wind                690107
date_time_y         690107
dtype: int64

In [64]:
# Check for null values in the merged datasets
cab.isnull().sum()

distance                 0
cab_type                 0
time_stamp_x             0
destination              0
source                   0
price                55095
surge_multiplier         0
id                       0
product_id               0
name                     0
date_time_x              0
m_date                   0
temp                  2964
location              2964
clouds                2964
pressure              2964
rain                594315
time_stamp_y          2964
humidity              2964
wind                  2964
date_time_y           2964
dtype: int64

In [65]:
# Handling missing values
cab = cab.dropna(subset='price')


weather_cols = ['temp', 'clouds', 'humidity', 'wind', 'pressure']
cab[weather_cols] = cab[weather_cols].fillna(method= 'ffill').fillna(method= 'bfill')

# for rain column fillna with 0
cab['rain'] = cab['rain'].fillna(0)

# 
cab['location'] = cab['location'].fillna(cab['source'])

C:\Users\Fhaarishhassan\AppData\Local\Temp\ipykernel_19304\3569439500.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cab[weather_cols] = cab[weather_cols].fillna(method= 'ffill').fillna(method= 'bfill')


In [66]:
cab.columns

Index(['distance', 'cab_type', 'time_stamp_x', 'destination', 'source',
       'price', 'surge_multiplier', 'id', 'product_id', 'name', 'date_time_x',
       'm_date', 'temp', 'location', 'clouds', 'pressure', 'rain',
       'time_stamp_y', 'humidity', 'wind', 'date_time_y'],
      dtype='object')

In [67]:
# Drop unnessary columns
cab = cab.drop(columns=['time_stamp_x', 'time_stamp_y', 'date_time_y', 'm_date'])
cab.columns

Index(['distance', 'cab_type', 'destination', 'source', 'price',
       'surge_multiplier', 'id', 'product_id', 'name', 'date_time_x', 'temp',
       'location', 'clouds', 'pressure', 'rain', 'humidity', 'wind'],
      dtype='object')

In [68]:
# Extract date-time features
cab['hour'] = cab['date_time_x'].dt.hour
cab['day'] = cab['date_time_x'].dt.day
cab['mounth'] = cab['date_time_x'].dt.month
cab['weekday'] = cab['date_time_x'].dt.weekday
cab['weekend'] = cab['date_time_x'].dt.weekday.isin([5,6]).astype(int)

# Drop date_time_x columns
cab = cab.drop(columns=['date_time_x'])

In [69]:
cab.columns

Index(['distance', 'cab_type', 'destination', 'source', 'price',
       'surge_multiplier', 'id', 'product_id', 'name', 'temp', 'location',
       'clouds', 'pressure', 'rain', 'humidity', 'wind', 'hour', 'day',
       'mounth', 'weekday', 'weekend'],
      dtype='object')

In [70]:
cab['id'].unique()
cab['product_id'].unique()

array(['lyft_line', 'lyft_premier', 'lyft', 'lyft_luxsuv', 'lyft_plus',
       'lyft_lux', '6f72dfc5-27f1-42e8-84db-ccc7a75f6969',
       '6c84fd89-3f11-4782-9b50-97c468b19529',
       '55c66225-fbe7-4fd5-9072-eab1ece5e23e',
       '9a0e7b09-b92b-4c41-9779-2ad22b4d779d',
       '6d318bcc-22a3-4af6-bddd-b409bfce1546',
       '997acbb5-e102-41e1-b155-9df7de0a73f2'], dtype=object)

In [71]:
# Verify if all source and location columns are same after filling missing values
(cab['source'] == cab['location']).all()

np.True_

In [72]:
cab = cab.drop(columns=['location', 'id', 'product_id' ])

In [73]:
# Reset index after droping rows
cab.reset_index(inplace=True)

In [74]:
cab.index

RangeIndex(start=0, stop=637976, step=1)

In [75]:
# Rename columns for better understanding
cab = cab.rename(columns={
    'distance': 'distance_km',
    'cab_type': 'company',
    'destination': 'drop_location',
    'source': 'pickup_location',
    'surge_multiplier': 'surge',
    'name': 'ride_type',
    'temp': 'temperature',
    'clouds': 'cloud_coverage',
    'rain': 'rainfall',
    'wind': 'wind_speed'
})

In [76]:
cab.columns

Index(['index', 'distance_km', 'company', 'drop_location', 'pickup_location',
       'price', 'surge', 'ride_type', 'temperature', 'cloud_coverage',
       'pressure', 'rainfall', 'humidity', 'wind_speed', 'hour', 'day',
       'mounth', 'weekday', 'weekend'],
      dtype='object')

In [77]:
cab.describe()

,index,distance_km,price,surge,temperature,cloud_coverage,pressure,rainfall,humidity,wind_speed,hour,day,mounth,weekday,weekend
count,637976.000000,637976.000000,637976.000000,637976.000000,637976.000000,637976.000000,637976.000000,637976.000000,637976.000000,637976.00000,637976.000000,637976.000000,637976.000000,637976.000000,637976.000000
mean,346446.048666,2.189261,16.545125,1.015068,39.222374,0.667874,1010.228828,0.005599,0.747333,6.91641,11.618528,17.797674,11.586550,2.781699,0.262076
std,200059.644995,1.135413,9.324359,0.095422,6.601370,0.313846,13.386430,0.026571,0.128962,3.64836,6.948776,9.982083,0.492452,2.056670,0.439764
min,0.000000,0.020000,2.500000,1.000000,19.620000,0.000000,988.250000,0.000000,0.450000,0.30000,0.000000,1.000000,11.000000,0.000000,0.000000
25%,173190.750000,1.270000,9.000000,1.000000,36.110000,0.430000,999.950000,0.000000,0.650000,3.63000,6.000000,13.000000,11.000000,1.000000,0.000000
50%,346400.000000,2.160000,13.500000,1.000000,40.180000,0.760000,1009.550000,0.000000,0.730000,6.74000,12.000000,17.000000,12.000000,3.000000,0.000000
75%,519730.250000,2.930000,22.500000,1.000000,43.600000,0.960000,1021.880000,0.000000,0.870000,9.88000,18.000000,28.000000,12.000000,5.000000,1.000000
max,693070.000000,7.860000,97.500000,3.000000,55.410000,1.000000,1035.120000,0.274000,0.990000,18.18000,23.000000,30.000000,12.000000,6.000000,1.000000


In [78]:
cab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 637976 entries, 0 to 637975
Data columns (total 19 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   index            637976 non-null  int64  
 1   distance_km      637976 non-null  float64
 2   company          637976 non-null  object 
 3   drop_location    637976 non-null  object 
 4   pickup_location  637976 non-null  object 
 5   price            637976 non-null  float64
 6   surge            637976 non-null  float64
 7   ride_type        637976 non-null  object 
 8   temperature      637976 non-null  float64
 9   cloud_coverage   637976 non-null  float64
 10  pressure         637976 non-null  float64
 11  rainfall         637976 non-null  float64
 12  humidity         637976 non-null  float64
 13  wind_speed       637976 non-null  float64
 14  hour             637976 non-null  int32  
 15  day              637976 non-null  int32  
 16  mounth           637976 non-null  int3

In [79]:
cab.to_csv("Cab_price_clean.csv",index=False)